In [1]:
import os
import json
from glob import glob
import pandas as pd
os.chdir("..")

In [2]:
json_list = glob('/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/*.json')
json_list

['/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/zodiac_sign.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/playlist_duration.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/sentiment_analysis.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/simple_interest.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/password_generator.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/date_difference.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/calculate_tips.json',
 '/Users/cs/Desktop/W210/function-calling-project/es_instruct/dump_json/difference_of_squares.json']

In [3]:
queries = []
answers = []
for file_path in json_list[0:]:
    with open(file_path, 'r') as file:
        text = json.load(file)
        text = text["queries"]
        for item in text:
            queries.append(item["query"])
            answers.append(json.dumps(item["answer"]))
        

In [4]:
import pandas as pd

# check schema 
df = pd.DataFrame({"queries": queries, "answers": answers})
df["functions"] = df["answers"].apply(lambda x: json.loads(x).get("function", None))


print(f"there are {len(df)} rows at the beginning")

there are 4820 rows at the beginning


In [5]:
#filtered non function
df = df[~df["functions"].isna()]

function_names = df["functions"].unique()

sample_list = []
for func_name in function_names:
    sample_df  = df[df["functions"]== func_name].sample(n=210, random_state=1)
    sample_list.append(sample_df)
df = pd.concat(sample_list)

print(f"there are {len(df)} after filtering out non function and sampling")


there are 1680 after filtering out non function and sampling


In [6]:
# filter arguments
df["arguments"] = df["answers"].apply(lambda x: json.loads(x).get("arguments", None))
df = df[~df["arguments"].isna()]
df["arguments"] = df["arguments"].apply(json.dumps)

print(f"there are {len(df)} has arguments keys")


there are 1680 has arguments keys


In [7]:
from src.chi_functions import *
from src.edward_functions import *
from src.MT_functions import * 
from src.kevin_function import * 
import swifter

def call_function(info: pd.DataFrame):
    try:
        func_to_call = eval(info["functions"])
        
        arguments = json.loads(info["arguments"])
        return func_to_call(**arguments)
    except:
        return None
df["callable"] = df.swifter.apply(lambda x: call_function(x), axis=1)
df = df[~df["callable"].isna()]

len(df)



Sentiment Analysis Result: {"result": {"polarity": 1.0, "subjectivity": 0.75}}
Difference of Squares Result: {"result": 84}
Generated Password: {"result": "Y33H?OFu&k;j0*#;"}
Interest result:  {"result": 400.0}


/Users/cs/Library/Caches/pypoetry/virtualenvs/function-calling-project-_d8EX7TE-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Pandas Apply: 100%|██████████| 1680/1680 [00:00<00:00, 24438.52it/s]


1680

In [51]:
# save DataFrame
df.reset_index(drop=True, inplace=True)
df.to_csv("df_checkpoint_1.csv", index=None)

print(df["functions"].unique())

df[df["functions"] == "playlist_duration_calculator"].iloc[4]["queries"]


['zodiac_sign' 'playlist_duration_calculator' 'sentiment_analysis'
 'simple_interest' 'password_generator' 'date_difference' 'calculate_tip'
 'difference_of_squares']


'Calculate the total playlist duration for 9 songs, average length 4.71437271016475 mins, include breaks: False, break duration: 1.11223561498185 mins.'

In [94]:
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import time
from tqdm import tqdm
import random

def generate_response(df, llm):
    query_output = df["queries"]
    function_output = df["callable"]
    messages = [
    ("system", f"Answer the question below in a complete sentence. Ensure your answer includes the key terms from the question, such as the subject or important descriptors. Here is the information that can help you answer the question: {function_output}. Do not generate a response more than 25 words"),
    ("user", f"{query_output}")]
    response = llm.invoke(messages)

    return response.content

retry_config = Config(
    retries = {
        'max_attempts': 20,  # Customize the number of retry attempts
        'mode': 'adaptive'   # Or 'adaptive' for dynamically adjusting retries
    }
)

# Initialize the Bedrock client with retry configuration
bedrock_client = boto3.client('bedrock-runtime', config=retry_config)

# Use the ChatBedrock model in LangChain with the client that has retries
llm = ChatBedrock(
    client=bedrock_client,
    model_id="meta.llama3-1-70b-instruct-v1:0",  # Or "meta.llama3-1-70b-instruct-v1:0"
    temperature=0.2,
    max_tokens=None)



responses = []
numbers = []
# os.remove("responses.csv"
# index = df.index.difference(response_df.index)
# for item in tqdm(df.iloc[index].iterrows()):
for item in tqdm(df.iterrows()):
    try:
        response = generate_response(item[1], llm)
        responses.append(response)
        numbers.append(item[0])
        time.sleep(random.random()*2)
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "responses": responses})
            temp_df.to_csv('responses.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
            

    except Exception as e:
        print(e)
        time.sleep(random.random()*100)
        response = generate_response(item[1], llm)
        responses.append(response)
        numbers.append(item[0])
        if item[0] % 5 ==0 :
            temp_df = pd.DataFrame({"index": numbers, "responses": responses})
            temp_df.to_csv('responses.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
        print(item[1]["queries"])
        
if item[0] % 5 != 0:
    temp_df = pd.DataFrame({"index": numbers, "responses": responses})
    temp_df.to_csv('responses.csv', mode='a', index=False, header=False)
    print("last set saved")


10it [00:21,  2.17s/it]


In [100]:
index = df.index.difference(response_df.index)
index
response_df.index[response_df.index.duplicated()]

Index([225], dtype='int64', name=0)

In [106]:
# merge response to the main dataframe
response_df = pd.read_csv("responses.csv", index_col=0, header=None)
response_df.rename(columns={1: "generated_response"}, inplace=True)
response_df.sort_index(inplace=True)
df = pd.read_csv("df_checkpoint_1.csv", index_col=None)
df["generated_responses"] = response_df["generated_response"]
df.to_csv("df_checkpoint_2.csv", index=None)



In [2]:
df = pd.read_csv("df_checkpoint_2.csv")
df

,queries,answers,functions,arguments,callable,generated_responses
0,What is the zodiac sign for someone born on 20...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,"{""birthday"": ""2021-04-28""}","{""result"": ""Taurus""}","The zodiac sign for someone born on April 28, ..."
1,What is the zodiac sign for someone born on 19...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,"{""birthday"": ""1987-09-20""}","{""result"": ""Virgo""}",The zodiac sign for someone born on September ...
2,What is the zodiac sign for someone born on 20...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,"{""birthday"": ""2003-09-16""}","{""result"": ""Virgo""}",The zodiac sign for someone born on 2003-09-16...
3,What is the zodiac sign for someone born on 19...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,"{""birthday"": ""1993-11-21""}","{""result"": ""Scorpio""}",The zodiac sign for someone born on November 2...
4,What is the zodiac sign for someone born on 19...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,"{""birthday"": ""1985-08-23""}","{""result"": ""Virgo""}","The zodiac sign for someone born on August 23,..."
...,...,...,...,...,...,...
1675,What is the difference of squares between 870....,"{""function"": ""difference_of_squares"", ""argumen...",difference_of_squares,"{""a"": 870.9055231201728, ""b"": 739.615958553147}","{""result"": 211444.66405473137}",The difference of squares between 870.90552312...
1676,What is the difference of squares between 256....,"{""function"": ""difference_of_squares"", ""argumen...",difference_of_squares,"{""a"": 256.1316994557745, ""b"": 667.9498901024173}","{""result"": -380553.6082217281}",The difference of squares is -380553.6082217281.
1677,What is the difference of squares between 628....,"{""function"": ""difference_of_squares"", ""argumen...",difference_of_squares,"{""a"": 628.2921580392243, ""b"": 525.4745800503505}","{""result"": 118627.50157449342}",The difference of squares between 628.29215803...
1678,What is the difference of squares between 74.2...,"{""function"": ""difference_of_squares"", ""argumen...",difference_of_squares,"{""a"": 74.20386222247328, ""b"": 101.43474783453082}","{""result"": -4782.794899523059}",The difference of squares between 74.203862222...


In [14]:
# reverse prompt engineering
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_community.utils.math import cosine_similarity
import numpy as np
import boto3
from langchain_aws import ChatBedrock
from botocore.config import Config
import time
from tqdm import tqdm
import random
import traceback



retry_config = Config(
    retries = {
        'max_attempts': 2,  # Customize the number of retry attempts
        'mode': 'adaptive'   # Or 'adaptive' for dynamically adjusting retries
    }
)

# Initialize the Bedrock client with retry configuration
bedrock_client = boto3.client('bedrock-runtime', config=retry_config)

# Use the ChatBedrock model in LangChain with the client that has retries

llm_reverse = ChatBedrock(
    client=bedrock_client,
    model_id="meta.llama3-1-70b-instruct-v1:0",  # Or "meta.llama3-1-70b-instruct-v1:0"
    temperature=0.2,
    max_tokens=None)

class Query(BaseModel):
    """reverse prompt engineering output"""
    questions: List[str] = Field(description="a list of questions according to the user repsonse")

parser = JsonOutputParser(pydantic_object=Query)

def get_reverse_prompt(n, df, llm):
    prompt = PromptTemplate(
        template="Generate " + str(n) + " most possible questions based on the answer. Please provide a JSON response strictly according to the schema without any additional comments or variations.\n{format_instructions}\n{answer}\n",
        input_variables=["answer"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    prompt_and_model = prompt | llm
    output = prompt_and_model.invoke({"answer": df["generated_responses"]})

    return parser.invoke(output)

responses = []
numbers = []
# os.remove("reverse_queries.csv")
for item in tqdm(df.iterrows()):
    try:
        reverse_queries = get_reverse_prompt(3, item[1], llm_reverse)
        queries_list = json.dumps(reverse_queries["questions"])
        responses.append(queries_list)
        numbers.append(item[0])
        time.sleep(random.random()*2)
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "reverse_queries": responses})
            temp_df.to_csv('reverse_queries.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
            
    except Exception as e:
        print(e)
        time.sleep(random.random()*70)
        reverse_queries = get_reverse_prompt(3, item[1], llm_reverse)
        queries_list = json.dumps(reverse_queries["questions"])
        responses.append(queries_list)
        numbers.append(item[0])
        time.sleep(random.random())
        if item[0] % 5 == 0 :
            temp_df = pd.DataFrame({"index": numbers, "reverse_queries": responses})
            temp_df.to_csv('reverse_queries.csv', mode='a', index=False, header=False)
            numbers = []
            responses = []
        print(item[1]["queries"])
        
if item[0] % 5 != 0:
    temp_df = pd.DataFrame({"index": numbers, "reverse_queries": responses})
    temp_df.to_csv('reverse_queries.csv', mode='a', index=False, header=False)
    print("last set saved")




0it [00:00, ?it/s]ERROR:root:Error raised by bedrock service: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 2): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 2): Too many requests, please wait before trying again. You have sent too many requests.  Wait before trying again.


0it [00:10, ?it/s]


KeyboardInterrupt: 

In [73]:
response_df = pd.read_csv("reverse_queries.csv", index_col=0, header=None)
response_df.rename(columns={1: "reverse_queries"}, inplace=True)
df = pd.read_csv("df_checkpoint_2.csv", index_col=None)
df["reverse_queries"] = response_df["reverse_queries"]
df.to_csv("df_checkpoint_3.csv", index=None)


In [81]:
import swifter
# calculate_simliarity
def calculate_mean_similiarity(df):
    model_id = "amazon.titan-embed-text-v2:0"
    # def get_cos_sim(query_response, reverse_queries):
    embedding = BedrockEmbeddings(model_id=model_id)
    
    
    original_embedded = embedding.embed_query(df["queries"])
    guess_embeded = embedding.embed_documents(json.loads(df["reverse_queries"]))
    return np.array([cosine_similarity([original_embedded], [item])[0] for item in guess_embeded]).mean()

df["mean_cosine_similarity"] = df.swifter.apply(calculate_mean_similiarity, axis=1)


/Users/cs/Library/Caches/pypoetry/virtualenvs/function-calling-project-_d8EX7TE-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Pandas Apply: 100%|██████████| 1680/1680 [39:33<00:00,  1.41s/it]


In [83]:
df.to_csv("df_checkpoint_4.csv")